In [ ]:

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np

train_dir = r'E:\Tensorflow\decision_tree\ChestXRay2017\chest_xray\train'
validation_dir = r'E:\Tensorflow\decision_tree\ChestXRay2017\chest_xray\test'
# Directory with our training horse pictures
train_PNEUMONIA_dir = os.path.join(r'E:\Tensorflow\decision_tree\ChestXRay2017\chest_xray\train\PNEUMONIA')

# Directory with our training human pictures
train_NORMAL_dir = os.path.join(r'E:\Tensorflow\decision_tree\ChestXRay2017\chest_xray\train\NORMAL')

train_PNEUMONIA_names = os.listdir(train_PNEUMONIA_dir)
print(train_PNEUMONIA_names[:10])

train_NORMAL_names = os.listdir(train_NORMAL_dir)
print(train_NORMAL_names[:10])

print('total training PNEUMONIA images:', len(os.listdir(train_PNEUMONIA_dir)))
print('total training NORMAL images:', len(os.listdir(train_NORMAL_dir)))

%matplotlib inline



# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_PNEUMONIA_pix = [os.path.join(train_PNEUMONIA_dir, fname) 
                for fname in train_PNEUMONIA_names[pic_index-8:pic_index]]
next_NORMAL_pix = [os.path.join(train_NORMAL_dir, fname) 
                for fname in train_NORMAL_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_PNEUMONIA_pix+next_NORMAL_pix):

  # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)

    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()



model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range=40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(200, 200),  # All images will be resized to 150x150
        batch_size=96,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size  = 32,
                                                         class_mode  = 'binary', 
                                                         target_size = (200, 200))

history = model.fit(train_generator,
                    steps_per_epoch=10,  
                    epochs=75,
                    validation_data = validation_generator,
                    validation_steps = 8,
                    verbose=1)

model.save("single_layer_model_95%.h5")
print("Saved model to disk")

import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

model = tf.keras.models.load_model('single_layer_model_95%.h5')


path=r'E:\Tensorflow\decision_tree\ChestXRay2017\chest_xray\test\NORMAL\NORMAL2-IM-0060-0001.jpeg'

img=image.load_img(path, target_size=(200, 200))
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=32)
print(classes[0])
if classes[0]>0.5:
    print("input image is normal")    
else:
    print("input image is infected")

[0.]
input image is infected
